In [1]:
import pandas as pd
import geopandas as gpd

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
gdf_br = gpd.read_file('data/censo_2010/BR_Setores_2021.zip')

In [3]:
df = pd.read_csv('resultados/modelo_preditivo.csv.zip')

/tmp/ipykernel_187035/1623530861.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('resultados/modelo_preditivo.csv.zip')


In [ ]:
# Apresentação do método

## Introdução
    # Objetivo rodar o modelo criado para 2010 a partir do modelo proposto pelo CEM
    # Aguardar os dados de 2022
## Método
    # Dimensões
        # as 3 dimensões que ficaram de fora
    # Modelos: LDA, RandomForest, SVM
    # agregados espacialmente: RM, UF e o restante MacroRegião
## Resultados
    # Modelo roda para todo BR
    # Performance de cada um dos modelos
    # percentual
    # Alguns exemplos
    # Resultado visual município de SP comparando com favelas, corticos, etc
## Próximos passos
    # Publicação de um Dash
        # por Município e RM
        # publicação de mapas e tabelas

